# Analysis of response data from BuzzFeed News / Ipsos Public Affairs survey on news platform trust

Note: Survey was fielded Jan. 12 & 13, 2017.

## Load and tidy-ify the data

In [1]:
import pandas as pd
import weightedcalcs

In [2]:
wc = weightedcalcs.Calculator("Weightvar")

In [3]:
responses = pd.read_csv("../data/survey-responses.csv")

In [4]:
USAGE_Qs = {
    "DYN101": "Social media (generally)",
    "DYN102": "Facebook",
    "DYN103": "Twitter",
    "DYN104": "Snapchat",
    "DYN105": "YouTube",
    "DYN106": "Online-only news publications",
    "DYN107": "Newspapers’ websites",
    "DYN108": "Cable news",
    "DYN109": "Broadcast TV news",
    "DYN110": "Print newspapers",
    "DYN111": "News radio",
    "DYN112": "Talk radio",
    "DYN113": "None of these",
}

In [5]:
tidy_usage = responses[list(USAGE_Qs.keys()) + [ "ID" ]]\
    .rename(columns=USAGE_Qs)\
    .pipe(pd.melt, id_vars=["ID"], var_name="platform", value_name="usage")
tidy_usage.head()

,ID,platform,usage
0,1035214017,Cable news,0
1,1143047381,Cable news,0
2,C1461636059,Cable news,1
3,C1461636326,Cable news,1
4,1148320559,Cable news,0


In [6]:
TRUST_Qs = {
    "GRID_DYN2_1_DYN2": "Social media (generally)",
    "GRID_DYN2_2_DYN2": "Facebook",
    "GRID_DYN2_3_DYN2": "Twitter",
    "GRID_DYN2_4_DYN2": "Snapchat",
    "GRID_DYN2_5_DYN2": "YouTube",
    "GRID_DYN2_6_DYN2": "Online-only news publications",
    "GRID_DYN2_7_DYN2": "Cable news",
    "GRID_DYN2_8_DYN2": "Broadcast TV news",
    "GRID_DYN2_9_DYN2": "Newspapers’ websites",
    "GRID_DYN2_10_DYN2": "Print newspapers",
    "GRID_DYN2_11_DYN2": "News radio",
    "GRID_DYN2_12_DYN2": "Talk radio",
}

In [7]:
tidy_trust = responses[list(TRUST_Qs.keys()) + [ "ID" ]]\
    .rename(columns=TRUST_Qs)\
    .pipe(pd.melt, id_vars=["ID"], var_name="platform", value_name="trust")
tidy_trust.head()

,ID,platform,trust
0,1035214017,Talk radio,5
1,1143047381,Talk radio,2
2,C1461636059,Talk radio,2
3,C1461636326,Talk radio,4
4,1148320559,Talk radio,2


In [8]:
tidy_responses = pd.merge(
    tidy_usage,
    tidy_trust,
    on=["ID", "platform"]
).set_index("ID").join(responses.set_index("ID")[["Weightvar"]])
tidy_responses["trust_mostly_or_always"] = tidy_responses["trust"].isin([ 1, 2 ])
tidy_responses["trust_rarely_or_never"] = tidy_responses["trust"].isin([ 4, 5 ])
tidy_responses.head()

,platform,usage,trust,Weightvar,trust_mostly_or_always,trust_rarely_or_never
ID,,,,,,
1001559731,Cable news,0,2,1.24412,True,False
1001559731,Talk radio,0,2,1.24412,True,False
1001559731,Newspapers’ websites,0,2,1.24412,True,False
1001559731,Social media (generally),0,4,1.24412,False,True
1001559731,News radio,0,2,1.24412,True,False


## Analyze results

### Gotten news through platform in past month

In [9]:
wc.mean(tidy_responses.groupby("platform"), "usage").sort_values(ascending=False).round(2)

platform
Broadcast TV news                0.56
Facebook                         0.55
Print newspapers                 0.39
Cable news                       0.38
Social media (generally)         0.33
Newspapers’ websites             0.33
News radio                       0.27
YouTube                          0.25
Online-only news publications    0.23
Twitter                          0.20
Talk radio                       0.17
Snapchat                         0.12
dtype: float64

### Trust: distribution of responses

In [10]:
wc.distribution(tidy_responses.groupby("platform"), "trust")\
    .rename(columns={
        1: "All of the time",
        2: "Most of the time",
        3: "About half the time",
        4: "Rarely",
        5: "Almost never",
        6: "Don’t know"
    }).sort_values("All of the time", ascending=False).round(2)

trust,All of the time,Most of the time,About half the time,Rarely,Almost never,Don’t know
platform,,,,,,
Broadcast TV news,0.15,0.44,0.28,0.07,0.04,0.02
Print newspapers,0.14,0.45,0.24,0.08,0.06,0.04
Newspapers’ websites,0.12,0.41,0.24,0.10,0.06,0.07
Cable news,0.12,0.38,0.27,0.10,0.06,0.07
YouTube,0.08,0.10,0.25,0.25,0.15,0.18
News radio,0.08,0.42,0.26,0.10,0.08,0.07
Facebook,0.07,0.11,0.30,0.27,0.17,0.08
Online-only news publications,0.06,0.29,0.33,0.14,0.07,0.10
Social media (generally),0.05,0.10,0.31,0.28,0.17,0.09


### Trust: "all of the time" + "most of the time"

In [11]:
wc.mean(tidy_responses.groupby("platform"), "trust_mostly_or_always")\
    .sort_values(ascending=False).round(2)

platform
Broadcast TV news                0.59
Print newspapers                 0.58
Newspapers’ websites             0.54
Cable news                       0.50
News radio                       0.50
Online-only news publications    0.35
Talk radio                       0.34
YouTube                          0.18
Facebook                         0.18
Social media (generally)         0.15
Twitter                          0.15
Snapchat                         0.10
dtype: float64

### Trust: "rarely" + "almost never"

In [12]:
wc.mean(tidy_responses.groupby("platform"), "trust_rarely_or_never")\
    .sort_values(ascending=False).round(2)

platform
Snapchat                         0.47
Social media (generally)         0.45
Facebook                         0.44
Twitter                          0.44
YouTube                          0.40
Talk radio                       0.24
Online-only news publications    0.21
News radio                       0.17
Cable news                       0.16
Newspapers’ websites             0.16
Print newspapers                 0.14
Broadcast TV news                0.12
dtype: float64

### Trust in news on platform *among people who said they had gotten news there*

In [13]:
wc.mean(tidy_responses[
    (tidy_responses["usage"] == 1)
].groupby("platform"), "trust_mostly_or_always")\
    .round(2).sort_values(ascending=False)

platform
Print newspapers                 0.74
Newspapers’ websites             0.69
News radio                       0.68
Broadcast TV news                0.66
Cable news                       0.65
Talk radio                       0.57
Online-only news publications    0.55
YouTube                          0.53
Twitter                          0.49
Snapchat                         0.47
Facebook                         0.27
Social media (generally)         0.25
dtype: float64

---

---

---